In [290]:
import numpy as np

def confidence_noise_determination(img, c, v, beta, r):
    """
    Parameters:
    M: deterministic mechanism
    D: data distribution
    m: sampling complexity
    c, v, beta: constants

    Returns:
    Gaussian covariance matrix ΣB
    """

    # Step 6: Calculate empirical mean and empirical covariance estimation
    mu_hat = np.mean(img, axis=0)
    empirical_cov = np.var(img, axis=0)

    # Step 7: Apply singular value decomposition (SVD)
    U, Lambda, UT = np.linalg.svd(empirical_cov)
    # print(Lambda)
    
    # Step 8: Determine the maximal index j0
    if c > np.max(Lambda):
        condition = False
    else:
        j0 = np.argmin([lambd for lambd in Lambda if lambd > c])
    
        # Step 9: Check the condition
        condition = np.min([np.abs(Lambda[j] - Lambda[l])for j in range(j0 + 1) for l in range(j + 1, len(Lambda))]) > r * np.sqrt(len(Lambda) * c) + 2 * c
    
    if condition:
        # Step 10 & 11: Determine the j-th element of a diagonal matrix ΛB
        sqrt_sum = sum(np.sqrt(Lambda[j] + 10 * c * v/ beta) for j in range(len(Lambda)))
        Lambda_B = [2 * v / (np.sqrt(Lambda[j] + 10 * c * v/ beta) * sqrt_sum) for j in range(len(Lambda))]
        ΛB = np.diag(Lambda_B)

        # Step 13: Determine the Gaussian noise covariance
        ΣB = U @ np.linalg.inv(ΛB) @ UT

    else:
        # Step 15: Determine the Gaussian noise covariance
        ΣB = (sum(Lambda) + len(Lambda) * c) / (2 * v) * np.eye(len(Lambda))

    return ΣB


In [291]:
import os
from PIL import Image, ImageOps 

In [292]:
img_dir = '/data/local/qipan/exp_celeba/fid_samples_larger_batch_2/images_exp_10/attractive/ckpt_300000'
img = [ImageOps.grayscale(Image.open(os.path.join(img_dir, f))) for f in os.listdir(img_dir)]
img = np.array([np.array(i) for i in img])

In [293]:
img.shape

(1000, 64, 64)

In [294]:
img = (img - np.min(img, axis=0))/ (np.max(img, axis=0)- np.min(img, axis=0))

In [295]:
empirical_cov = np.var(img, axis=0)

In [296]:
empirical_cov.shape

(64, 64)

In [297]:
empirical_cov

array([[0.10783343, 0.10652024, 0.10708549, ..., 0.11586227, 0.1163765 ,
        0.11867911],
       [0.10817664, 0.10570074, 0.10601614, ..., 0.11445619, 0.11440364,
        0.11613441],
       [0.10759764, 0.10560437, 0.10402779, ..., 0.11260253, 0.11430152,
        0.11625804],
       ...,
       [0.10570882, 0.10178702, 0.09829975, ..., 0.10656897, 0.10844319,
        0.1101314 ],
       [0.10409217, 0.10076563, 0.098434  , ..., 0.10646031, 0.10824437,
        0.10930396],
       [0.10423785, 0.10187556, 0.09987599, ..., 0.1077231 , 0.10871629,
        0.11174058]])

In [298]:
import numpy as np

# This is a placeholder for the actual sub-Gaussian norm computation.
def subgaussian_norm(X):
    # You would replace this with the actual calculation, which depends on the distribution of X.
    # For example, for a standard normal variable, the sub-Gaussian norm is 1.
    return 1

def calculate_K(A):
    # Assuming A is a 2D numpy array of random variables.
    subgaussian_norms = np.array([[subgaussian_norm(A[i, j]) for j in range(A.shape[1])] for i in range(A.shape[0])])
    K = np.max(subgaussian_norms)
    return K

# Example usage:
# A = np.random.randn(m, n)  # This would be your m x n random matrix with sub-Gaussian entries.
# K = calculate_K(A)
# print(K)

In [299]:
calculate_K(img[0])

1

In [300]:
d = 64
gamma = 0.5
m = img.shape[0]
r = np.max(np.linalg.norm(img, axis=(1,2)))
print(r)
c = r * (np.max([np.sqrt((d + np.log(4/gamma))/m), (d + np.log(4/gamma))/m]) + np.sqrt((d + np.log(4/gamma))/m))
c

56.51987662029019


29.057910567178592

In [301]:
# c = 1
v = 0.5
beta = 0.5
ΣB = confidence_noise_determination(img, c, v, beta, r)

In [302]:
ΣB

array([[1865.42122263,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        , 1865.42122263,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        , 1865.42122263, ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ..., 1865.42122263,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
        1865.42122263,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        , 1865.42122263]])

In [303]:
ΣB.shape

(64, 64)

In [304]:
B = np.random.multivariate_normal(np.zeros(d), ΣB, m)

In [305]:
B

array([[-2.90755080e+01, -5.58307464e+00,  5.68848965e+01, ...,
         8.84992789e+00, -4.29453491e+01,  2.98019342e+01],
       [-3.09257814e-01, -7.54811290e+01, -1.40987420e+01, ...,
         2.33519696e+01, -9.88309215e+00, -1.59748082e+01],
       [ 1.24842973e+01, -1.84669163e+01,  7.71649799e-02, ...,
         7.19798498e+01,  4.59582348e+00, -5.82638597e+01],
       ...,
       [ 7.50279147e+01, -3.95651156e+01,  3.90375089e+01, ...,
        -1.73166254e+01, -1.04513898e+02,  4.10215176e+01],
       [ 5.39807790e+00, -3.64947818e+01, -2.72661970e+00, ...,
         1.65864998e+00,  3.24644478e+01,  2.96363470e+01],
       [ 1.38758207e+01, -1.94302140e+01,  2.37708890e+01, ...,
        -3.37397871e+01, -1.85242238e+01,  3.27796217e+01]])

In [306]:
img[0] + B[0]

array([[-28.1264884 ,  -4.63405503,  57.84175929, ...,   9.84600632,
        -41.95319224,  30.79409103],
       [-28.11864527,  -4.63013346,  57.82215145, ...,   9.84208475,
        -41.95711381,  30.79409103],
       [-28.13433154,  -4.64581974,  57.79470047, ...,   9.84208475,
        -41.95711381,  30.79016946],
       ...,
       [-28.07550801,  -4.58699621,  57.85744557, ...,   9.83032005,
        -41.96495695,  30.77840475],
       [-28.07550801,  -4.58307464,  57.87705341, ...,   9.83032005,
        -41.96887852,  30.77840475],
       [-28.07550801,  -4.58307464,  57.87313184, ...,   9.83032005,
        -41.96887852,  30.77840475]])

In [307]:
np.mean(np.linalg.norm(B, axis=1))

344.0410737302445